In [11]:
# !pip install requests
# !pip install bs4
!pip install pandas

In [2]:
url = 'https://www.hemnet.se/salda/bostader?living_area_max=45&living_area_min=35&location_ids%5B%5D=473422&page=2&rooms_min=1.5'

In [5]:
import requests

def load_html(url):
    headers = {
        'User-Agent': 'Your User Agent String'  # Replace with an appropriate User-Agent
    }

    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        return response.text
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
        return None

if __name__ == "__main__":
    url = input("Enter the URL: ")
    html_content = load_html(url)

    if html_content:
        print(html_content)
    else:
        print("Failed to fetch HTML content.")


<!DOCTYPE html>
<!--[if IE 8 ]>    <html lang="sv-SE" dir="ltr" class="ie8"> <![endif]-->
<!--[if IE 9 ]>    <html lang="sv-SE" dir="ltr" class="ie9"> <![endif]-->
<!--[if (gt IE 9)|!(IE)]><!--> <html lang="sv-SE" dir="ltr"> <!--<![endif]-->
  <!--
           hhhhhhhh
      hhhhhhhhhhhhhhhhhh
    hhhhh             hhhh
  hhhhh                 hhhh
 hhhh         hhh        hhhh
 hhh       hhhhhhhhh      hhhh
hhhh     hhhhhhhhhhhh     hhhh
hhhh     hhhh    hhhh     hhhh
hhhh     hhhh    hhhh     hhhh
 hhhh                    hhhh
  hhhhh                 hhhh
   hhhhhh            hhhhh
 hhhhh hhhhhhhhhhhhhhhh
hhhh     hhhhhhhhhhh

Kul att du hittat hit! Vill du också jobba på en av Sveriges största sajter med miljontals unika besökare?
Gör en spontanansökan på https://career.hemnet.se/
-->

  <head>
    
<meta charset="utf-8">
<link rel="dns-prefetch" href="//assets.hemnet.se">
<link rel="dns-prefetch" href="//bilder.hemnet.se">
<link rel="dns-prefetch" href="//maps.googleapis.com">
<link

In [ ]:
html_response = html_content


In [30]:
import pandas as pd
from bs4 import BeautifulSoup

def extract_rooms(room_text):
    if "rum" in room_text:
        room_parts = room_text.split(" ")
        for part in room_parts:
            if "rum" in part:
                rooms = part.replace(",", ".").replace("&nbsp;", "").replace("rum", "")
                try:
                    return float(rooms)
                except ValueError:
                    return 0.0
    return 0.0

def extract_values_from_html(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')

    results = []

    listings = soup.find_all('li', class_='sold-results__normal-hit')
    for listing in listings:
        data_element = listing.find('span', class_='hcl-label--sold-at')
        data = data_element.text.strip() if data_element else ""

        address_element = listing.find('h2', class_='sold-property-listing__heading')
        address = address_element.text.strip() if address_element else ""

        avgift_element = listing.find('div', class_='sold-property-listing__fee')
        avgift_text = avgift_element.text.strip() if avgift_element else ""
        try:
            avgift = int(avgift_text.replace(" ", "").replace("\xa0", "").replace("kr/mån", "")) if avgift_text else 0
        except ValueError as avgift_error:
            print(f"Error extracting avgift: {avgift_error}")
            avgift = avgift_text

        slutpris_element = listing.find('span', class_='hcl-text hcl-text--medium')
        slutpris_text = slutpris_element.text.strip() if slutpris_element else ""
        try:
            slutpris = int(slutpris_text.replace(" ", "").replace("\xa0", "").replace("kr", "").replace("Slutpris", "")) if slutpris_text else 0
        except ValueError as slutpris_error:
            print(f"Error extracting slutpris: {slutpris_error}")
            slutpris = slutpris_text

        sqm_pris_element = listing.find('div', class_='sold-property-listing__price-per-m2')
        sqm_pris_text = sqm_pris_element.text.strip() if sqm_pris_element else ""
        try:
            sqm_pris = int(sqm_pris_text.replace(" ", "").replace("\xa0", "").replace("kr/m²", "")) if sqm_pris_text else 0
        except ValueError as sqm_pris_error:
            print(f"Error extracting sqm_pris: {sqm_pris_error}")
            sqm_pris = sqm_pris_text

        name_element = listing.find('span', class_='sold-property-listing__first')
        name = name_element.text.strip() if name_element else ""

        size_element = listing.find('div', class_='sold-property-listing__area')
        size_text = size_element.text.split('\n')[1].strip()
        try:
            size_parts = size_text.split()
            size_numeric = float(size_parts[0].replace(",", ".")) if size_parts[0] else 0.0
            rooms = extract_rooms(size_text)
        except ValueError as size_error:
            print(f"Error extracting size: {size_error}")
            size_numeric = size_text
            rooms = 0.0

        results.append({
            "data": data,
            "address": address,
            "avgift": avgift,
            "slutpris": slutpris,
            "sqm_pris": sqm_pris,
            "name": name,
            "size": size_numeric,
            "rooms": rooms
        })

    return results

# Replace this with your actual HTML response
html_response = html_content

# Call the function and create a pandas DataFrame
extracted_values = extract_values_from_html(html_response)
df = pd.DataFrame(extracted_values)

# Print the DataFrame
print(df)


                      data                               address  avgift  \
0    Såld 28 december 2021                      Flottbrovägen 27    2913   
1     Såld 3 december 2021                 Essingeringen 15, 3tr    2132   
2    Såld 24 november 2021                     Essingestråket 21    1596   
3    Såld 16 november 2021           Badstrandsvägen 22, 3 / 3tr    1829   
4    Såld 16 november 2021               Essingestråket 11, 2 tr    3570   
5    Såld 16 november 2021                      Essingetorget 40    2946   
6    Såld 11 november 2021                    Badstrandsvägen 31    1725   
7     Såld 30 oktober 2021              Badstrandsvägen 20, 2 tr    2189   
8     Såld 28 oktober 2021                        Eknäsvägen 6 C    2295   
9     Såld 23 oktober 2021                        Eknäsvägen 6 B    2544   
10    Såld 22 oktober 2021                Essingestråket 11, 3tr    3230   
11    Såld 11 oktober 2021                      Essingetorget 46    2053   
12     Såld 

In [57]:
df = pd.DataFrame()

# First call outside the loop to initialize the DataFrame structure
extracted_values = extract_values_from_html(load_html('https://www.hemnet.se/salda/bostader?living_area_max=45&living_area_min=35&location_ids%5B%5D=473422&rooms_min=1.5'))
df = df.append(extracted_values, ignore_index=True)

# Loop to call the function and add values
for i in range(6):  # Replace 10 with the desired number of iterations
    extracted_values = extract_values_from_html(load_html(urls[i]))
    df = df.append(extracted_values, ignore_index=True)

# Print the final DataFrame
print(df)

/var/folders/n4/37zl64v150z2lq9vhml1q6lh0000gn/T/ipykernel_2128/1349945559.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(extracted_values, ignore_index=True)
/var/folders/n4/37zl64v150z2lq9vhml1q6lh0000gn/T/ipykernel_2128/1349945559.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(extracted_values, ignore_index=True)
/var/folders/n4/37zl64v150z2lq9vhml1q6lh0000gn/T/ipykernel_2128/1349945559.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(extracted_values, ignore_index=True)
/var/folders/n4/37zl64v150z2lq9vhml1q6lh0000gn/T/ipykernel_2128/1349945559.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pa

                      data                 address  avgift  slutpris  \
0     Såld 13 augusti 2023      Badstrandsvägen 27    2760   3300000   
1        Såld 21 juli 2023       Essingestråket 23    1470   3200000   
2         Såld 7 juli 2023       Stenkullavägen 54    2006   3050000   
3         Såld 7 juli 2023       Essingestråket 37    1641   3300000   
4        Såld 14 juni 2023        Flottbrovägen 15    3160   3045000   
..                     ...                     ...     ...       ...   
337     Såld 15 april 2013   Stenkullavägen 48 1tr    2060   1950000   
338      Såld 28 mars 2013        Essingetorget 44    2164   2030000   
339      Såld 15 mars 2013   Essingestråket 32 2tr    3368   2300000   
340      Såld 15 mars 2013  Essingestråket 11, 2tr    2856   2000000   
341  Såld 28 februari 2013   Essingestråket 44 6tr    1552   2305000   

     sqm_pris     name  size  rooms  
0       76744  Kerstin  43.0    0.0  
1       91429   Martin  35.0    0.0  
2       80263        

/var/folders/n4/37zl64v150z2lq9vhml1q6lh0000gn/T/ipykernel_2128/1349945559.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(extracted_values, ignore_index=True)


In [49]:
urls = [
    'https://www.hemnet.se/salda/bostader?living_area_max=45&living_area_min=35&location_ids%5B%5D=473422&page=2&rooms_min=1.5',
    'https://www.hemnet.se/salda/bostader?living_area_max=45&living_area_min=35&location_ids%5B%5D=473422&page=3&rooms_min=1.5',
    'https://www.hemnet.se/salda/bostader?living_area_max=45&living_area_min=35&location_ids%5B%5D=473422&page=4&rooms_min=1.5',
    'https://www.hemnet.se/salda/bostader?living_area_max=45&living_area_min=35&location_ids%5B%5D=473422&page=5&rooms_min=1.5',
    'https://www.hemnet.se/salda/bostader?living_area_max=45&living_area_min=35&location_ids%5B%5D=473422&page=6&rooms_min=1.5',
    'https://www.hemnet.se/salda/bostader?living_area_max=45&living_area_min=35&location_ids%5B%5D=473422&page=7&rooms_min=1.5',
    'https://www.hemnet.se/salda/bostader?living_area_max=45&living_area_min=35&location_ids%5B%5D=473422&page=8&rooms_min=1.5',
    'https://www.hemnet.se/salda/bostader?living_area_max=45&living_area_min=35&location_ids%5B%5D=473422&page=9&rooms_min=1.5',
    'https://www.hemnet.se/salda/bostader?living_area_max=45&living_area_min=35&location_ids%5B%5D=473422&page=10&rooms_min=1.5',
    'https://www.hemnet.se/salda/bostader?living_area_max=45&living_area_min=35&location_ids%5B%5D=473422&page=11&rooms_min=1.5',
]

In [58]:
df

,data,address,avgift,slutpris,sqm_pris,name,size,rooms
0,Såld 13 augusti 2023,Badstrandsvägen 27,2760,3300000,76744,Kerstin,43.0,0.0
1,Såld 21 juli 2023,Essingestråket 23,1470,3200000,91429,Martin,35.0,0.0
2,Såld 7 juli 2023,Stenkullavägen 54,2006,3050000,80263,,38.0,0.0
3,Såld 7 juli 2023,Essingestråket 37,1641,3300000,94286,,35.0,0.0
4,Såld 14 juni 2023,Flottbrovägen 15,3160,3045000,80132,Emma,38.0,0.0
...,...,...,...,...,...,...,...,...
337,Såld 15 april 2013,Stenkullavägen 48 1tr,2060,1950000,51316,,38.0,0.0
338,Såld 28 mars 2013,Essingetorget 44,2164,2030000,47209,,43.0,0.0
339,Såld 15 mars 2013,Essingestråket 32 2tr,3368,2300000,51225,,44.9,0.0
340,Såld 15 mars 2013,"Essingestråket 11, 2tr",2856,2000000,47619,,42.0,0.0


In [39]:
df.to_excel('stora_essingen_40.xlsx')

In [59]:
import re
import pandas as pd


# Define a function to extract and convert the date string
def extract_date(input_text):
    try : 
        match = re.search(r"Såld (\d+) (\w+) (\d+)", input_text)
        day = match.group(1)
        month = match.group(2)
        year = match.group(3)

        months_dict = {
            "januari": "1",
            "februari": "2",
            "mars": "3",
            "april": "4",
            "maj": "5",
            "juni": "6",
            "juli": "7",
            "augusti": "8",
            "september": "9",
            "oktober": "10",
            "november": "11",
            "december": "12",
        }

        month_numeric = months_dict.get(month.lower(), "00")
        date_string = f"{day}.{month_numeric}.{year}"
        return date_string
    except :
        return "input_text"

# Apply the function to the 'data' column
df['date_mod'] = df['data'].apply(extract_date)

print(df)


                      data                 address  avgift  slutpris  \
0     Såld 13 augusti 2023      Badstrandsvägen 27    2760   3300000   
1        Såld 21 juli 2023       Essingestråket 23    1470   3200000   
2         Såld 7 juli 2023       Stenkullavägen 54    2006   3050000   
3         Såld 7 juli 2023       Essingestråket 37    1641   3300000   
4        Såld 14 juni 2023        Flottbrovägen 15    3160   3045000   
..                     ...                     ...     ...       ...   
337     Såld 15 april 2013   Stenkullavägen 48 1tr    2060   1950000   
338      Såld 28 mars 2013        Essingetorget 44    2164   2030000   
339      Såld 15 mars 2013   Essingestråket 32 2tr    3368   2300000   
340      Såld 15 mars 2013  Essingestråket 11, 2tr    2856   2000000   
341  Såld 28 februari 2013   Essingestråket 44 6tr    1552   2305000   

     sqm_pris     name  size  rooms    date_mod  
0       76744  Kerstin  43.0    0.0  13.08.2023  
1       91429   Martin  35.0    0.0

In [60]:
df

,data,address,avgift,slutpris,sqm_pris,name,size,rooms,date_mod
0,Såld 13 augusti 2023,Badstrandsvägen 27,2760,3300000,76744,Kerstin,43.0,0.0,13.08.2023
1,Såld 21 juli 2023,Essingestråket 23,1470,3200000,91429,Martin,35.0,0.0,21.07.2023
2,Såld 7 juli 2023,Stenkullavägen 54,2006,3050000,80263,,38.0,0.0,7.07.2023
3,Såld 7 juli 2023,Essingestråket 37,1641,3300000,94286,,35.0,0.0,7.07.2023
4,Såld 14 juni 2023,Flottbrovägen 15,3160,3045000,80132,Emma,38.0,0.0,14.06.2023
...,...,...,...,...,...,...,...,...,...
337,Såld 15 april 2013,Stenkullavägen 48 1tr,2060,1950000,51316,,38.0,0.0,15.04.2013
338,Såld 28 mars 2013,Essingetorget 44,2164,2030000,47209,,43.0,0.0,28.03.2013
339,Såld 15 mars 2013,Essingestråket 32 2tr,3368,2300000,51225,,44.9,0.0,15.03.2013
340,Såld 15 mars 2013,"Essingestråket 11, 2tr",2856,2000000,47619,,42.0,0.0,15.03.2013


In [62]:
df.to_excel('v2.xlsx')